<a id='logbook'></a>
# Prediction of roll damping using fully nonlinear potential flow and Ikeda's method

In [1]:
# %load ../../notebooks/imports.py
"""
These is the standard setup for the notebooks.
"""

%matplotlib inline
%load_ext autoreload
%autoreload 2

from jupyterthemes import jtplot
jtplot.style(theme='onedork', context='notebook', ticks=True, grid=False)

import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option("display.max_columns", None)
import numpy as np
import os
import matplotlib.pyplot as plt
from collections import OrderedDict
#plt.style.use('paper')

#import data
import copy
from mdldb.run import Run

from sklearn.pipeline import Pipeline
from rolldecayestimators.transformers import CutTransformer, LowpassFilterDerivatorTransformer, ScaleFactorTransformer, OffsetTransformer
from rolldecayestimators.direct_estimator_cubic import EstimatorQuadraticB, EstimatorCubic
from rolldecayestimators.ikeda_estimator import IkedaQuadraticEstimator
import rolldecayestimators.equations as equations
import rolldecayestimators.lambdas as lambdas
from rolldecayestimators.substitute_dynamic_symbols import lambdify
import rolldecayestimators.symbols as symbols
import sympy as sp

from sympy.physics.vector.printing import vpprint, vlatex
from IPython.display import display, Math, Latex

from sklearn.metrics import r2_score
from src.data import database
from mdldb import tables
import shipflowmotionshelpers.shipflowmotionshelpers as helpers


Duplicate key in file WindowsPath('C:/Users/maa/.matplotlib/stylelib/paper.mplstyle'), line 461 ('figure.figsize   : 5, 3   ## figure size in inches')
Duplicate key in file WindowsPath('C:/Users/maa/.matplotlib/stylelib/paper.mplstyle'), line 462 ('figure.dpi       : 100        ## figure dots per inch')


# Abstract

# Nomenclature
| Variable | Explain |
|---|---|
|$\pi$| example |

# Introduction
Many great semi-empirical methods have been developed over the years to analyze various aspects of ship hydrodynamics such as:  resistance,  propulsion  and  seakeeping. These methods were often developed because solving the actual flow was far to complicated and far to time consuming at the time. When computers are now faster and great advancement in field of Computation Fluid Dynamics (CFD) has been achieved, the simpler semi-empirical formulas have now gotten less and less relevant. But have they all of gotten totally irrelevant? 

A semi-empirical method to predict ship roll damping, comonly known as Ikeda's method, will be investigated in this paper, for a use case that can still make it relevant. It will be investigated if this method can be used to increase the accuracy of the roll motion prediction of a nonlinear potential flow method (FNPF), so that this relatively fast option can be used to a larger extent than what is currently possible today. The roll damping of ship is higly depending on vicsous effects which mean that the invicid potetntial flow lacks a lot of the important physics of the roll motion and can therefore not give relevant results for this degree of freedom, where more advanced options such as: model test or URANS is needed.  

The hybrid method, combining Ikeda's method and FNPF proposed in this paper is investigated using the well known KVLCC2 test case ([Body plan](../../notebooks/11.1_KVLCC2_geometry.ipynb#bodyplan)). 
This ship was selected partly because it is a well known test case, there were model tests data available from SSPA and also because it does not have any bilge keels. The Ikeda's metod contain methods to predict damping from various component, where the bilge keels is one of them. From the model tests, only the total damping can be observed. Reducing the number of components by having no bilge keels would therefore give more insight into the remaining ones. 

Revisinging these older semi-empirical methods can also help us to understand the hydrodynamics better.

# Methods

## PIT

The initial heel angle during the roll decay tests gives the ship potential energy that is shifting to kinetic energy as the ship starts to move during the inital phase of the roll decay test. The energy is transfered between kinetic energy and potential energy during the oscillations. The ship loses energy over time due to the damping wich can be seen in this [plot](../../notebooks/02.2_ikeda_Be_assumption.ipynb#energy)

## Ikeda's method

### Eddy damping
The current implementation was benchmarked with the data from Ikeda's oringinal [cylinder experiments](../../notebooks/08.1_ikeda_eddy.ipynb). It seems that the current implementation underpredicts the damping.

![](../../notebooks/ikeda_eddy_section_A.PNG)

## FNPF

## Combined method

# Results

## Roll decay tests
### 0 knots
Data from two roll decay model tests conducted at zero speed were available: [Roll decay model tests 0 knots](../../notebooks/01.2_select_suitable_MDL_test_KLVCC2.ipynb#rolldecay). These tests where analyzed by fitting a [Cubic model](../../notebooks/01.2_select_suitable_MDL_test_KLVCC2.ipynb#cubic_model) to the model test data. The two models were very similar in terms of roll damping and stiffness, suggesting good repeatability in the model tests as well as in the parameter identification technique (PIT) used. The total damping at zero speed is very low for this vessel, where it takes at least one hundred oscillations for the motion to decay. Since the ship has no bilge keel, the friction damping and eddy damping will be the only damping contributions acting. These contributions are also rather small.   

Roll decay test was also simulated using [FNPF](../../notebooks/06.1_KVLCC2_motions.ipynb#damping). The damping was much higher than the predicted valued using Strip Theory.

### 15.5 knots
Data from one roll decay model tests conducted at a ship speed corresponding to 15.5 knots full scale ship speed was also available. This model tests was analyzed in the same way as the other tests at zero knots. It was found that the damping was higher at speed. The ship got a small [yaw rate](../../notebooks/01.3_select_suitable_MDL_test_KLVCC2_speed.ipynb#yawrate) at the end of test, giving a small steady roll angle due to the centrifugal force. Since this effect is not included in the matematical model used, the steady roll angle was instead removed by removing the linear trend in the roll angle signal. 




## Roll damping prediction with Ikeda's method
Calculated the roll damping with Ikeda's method for zero speed and 15.5 knots. The damping was [compared](../../notebooks/04.3_KVLCC2_Ikedas_model_tests.ipynb#speeds) with corresponding values obtained from the model tests. The results showed good agreement between the model tests and predictions at speed. The damping predicted with Ikeda's method at zero knots was significantly higher than the corresponding model test. The predicted eddy damping was very large and is most likey overpredicted.  

Much effort was made to investigate the eddy damping prediction of the Ikeda's method implementation. It was found, by conducting a sensitivity analysis, that the predicted eddy damping was extremely sensitive to ship sections with almost rectangular shape, where the section area coefficient [sigma](../../notebooks/08.2_ikeda_eddy_sigma.ipynb) was close to one. According to Kawahara the mid section coefficient should not exceed 0.99, which otherwise seem to give a huge extrapolation error according to the [figure](../../notebooks/08.2_ikeda_eddy_sigma.ipynb) above. This was however the case for many of the sections of the KVLCC2, where the limiting value was instead used, to avoid unrealistic extrapolation errors.

It seems that mid section coefficient most often exceeds 0.99 based on statistics from MDL tests during the past 15 years: [plot](../../notebooks/09.1_sigma_statistics.ipynb)

Eddy damping gets smaller as the speed increases and has almost vanished for the 15.5 knots which can be seen in [plot](../../notebooks/06.1_KVLCC2_motions.ipynb#speed). So the difficulties with eddy damping prediction for high mid section coefficients is mostly a low speed problem; And in the presens of bilge keels, the bilge keel damping would be the dominating contribution to the roll damping at low speed as can be seen in <cite data-cite="7505983/FJHQJJUH"></cite>. So this is mostly a problem at low speed for ships without bilge keels.

Attempts where made to use Ikeda's alternative eddy damping formula for barges. This one did not work the $B_E$ is far to large: [07.1_ikeda_barge](../../notebooks/07.1_ikeda_barge.ipynb)


## Combined method
Roll damping was predicted using the [Combined method](../../notebooks/06.1_KVLCC2_motions.ipynb#combined_damping). The damping witht the combined method seems to underpredict the damping at zero speed.

# Conclusions

# References
<div class="cite2c-biblio"></div>